In [35]:
import numpy as np
import cv2
import ntpath

# taking image path or name from user

path = input('Enter Image path to the binary image: ')
image = cv2.imread(path)

# extracting name of image file from path

name = ntpath.basename(path)
name = name.replace('.png','')

# Applying Erosion and Dilation operations
k3 = np.ones((3,3), np.uint8)
k5 = np.ones((5,5), np.uint8)
k13 = np.ones((13,13), np.uint8)

img1 = image.copy()
img1 = cv2.erode(img1, k3, iterations = 1)
img1 = cv2.dilate(img1, k3, iterations = 1)
    
cv2.imwrite(name+"-blobs.png", img1)

#separating border contour with the rest of image
for i in range(img1.shape[1]):
    img1[0,i] = 255
    img1[img1.shape[0]-1,i] = 255
    
for j in range(img1.shape[0]):
    img1[j,0] = 255
    img1[j,img1.shape[1]-1] = 255

# finding contours
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)   
contours, hierarchy = cv2.findContours(img1_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# drawing contours with random colors
img2 = img1.copy()
for i in range(len(contours)):
    color = (np.random.randint(256),np.random.randint(256),np.random.randint(256))
    cv2.drawContours(img2, contours, i, color, 2)

cv2.imwrite(name+"-contours.png", img2)

# storing areas of all contours
areas = []
for i in range(len(contours)):
    areas.append(cv2.contourArea(contours[i]))
areas = np.array(areas)
threshold_area = 1000

# calculating number of cracks
crack_num = np.shape(np.where(areas > threshold_area))[1]

# drawing cracks
img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img3 = np.zeros(img_gray.shape,dtype=np.uint8)
img3.fill(255)
crack_contour_indices = np.where(areas > threshold_area)[0]
for k in range(1, crack_num):
    cv2.drawContours(img3, contours, crack_contour_indices[k], (0,0,0), 1)

# Thinning Process :-    

# Structuring Element
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))

# closing contours with erosion
img3 = cv2.bitwise_not(img3)
img3 = cv2.morphologyEx(img3,cv2.MORPH_CLOSE,k13)

# # replicating only those pixels in the original image that are part of recognized cracks
# img4 = cv2.bitwise_not(img_gray)
# output = cv2.bitwise_and(img3, img4)
# output = cv2.bitwise_not(output) # inverting the colors

# cv2.imwrite(name+"-cracks.png", output)

img4 = img3.copy()

# Create an empty output image to hold values
output = np.zeros(img4.shape,dtype='uint8')
 
# Loop until erosion leads to an empty set
while (cv2.countNonZero(img4)!=0):
    # Erosion
    erode = cv2.erode(img4,kernel)
    # Opening on eroded image
    opening = cv2.morphologyEx(erode,cv2.MORPH_OPEN,kernel)
    # Subtract these two
    subset = erode - opening
    # Union of all previous sets
    output = cv2.bitwise_or(subset,output)
    # Set the eroded image for next iteration
    img4 = erode.copy()
    
output = cv2.bitwise_not(output)

cv2.imwrite(name+"-cracks.png", output)

# showing images
cv2.imshow("input", image)
cv2.imshow("blobs", img1)
cv2.imshow("contours", img3)
cv2.imshow("cracks", output)

cv2.waitKey(0)
cv2.destroyAllWindows()

Enter Image path to the binary image: wall2.png
